In [3]:
from pathlib import Path
from tqdm import tqdm
import time
import pandas as pd
import os
import numpy as np
import requests
from bs4 import BeautifulSoup
import openpyxl
import warnings

from matplotlib import pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

In [40]:
url_base = 'https://www.dane.gov.co'
url = 'https://www.dane.gov.co/index.php/estadisticas-por-tema/agropecuario/sistema-de-informacion-de-precios-sipsa/mayoristas-boletin-semanal-1'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36"
}
response = requests.get(url, headers = headers)
if response.status_code == 200:
  soup = BeautifulSoup(response.content, "html.parser")
  
  # Getting link for each year
  link_years = soup.find_all(lambda tag: tag.name == 'a' and any(substring in tag.get('title', '').lower() for substring in ['boletín mayorista semanal','boletin mayorista semanal', 'mayoristas boletín semanal']))

  # within each year
  for link in link_years:
      
      print(f' Working on {link.text.strip()} data')

      # Request for reports available in such year
      r = requests.get(url_base + link['href'], headers = headers)
      soup_year = BeautifulSoup(r.content, "html.parser")
      
      # Create links for each report within the given year
      target_links = [item for item in soup_year.find_all(target = '_blank') if 'Anexo' in item.text]
      date_elements = soup_year.find_all(lambda tag: tag.name == 'td' and tag.find('img', src="/files/images/boton.png")) #
      

      n = len(target_links)

      # Create folders per year
      REPORTS_PATH = Path.cwd()/'reports'/link.text.strip()
      if REPORTS_PATH.exists():
          print(f'{REPORTS_PATH} already exists')
      else: 
          REPORTS_PATH.mkdir(parents= True, exist_ok=True)

      
      # For each link, create file in local and fill it with data 
      for i, file in tqdm(enumerate(target_links)):
          # print(file)
          # request information
          try: 
              result = requests.get(url_base+file['href'], headers = headers)
          except:
              result = requests.get(file['href'], headers = headers)
          
          # name output file
          file_name = 'week_'+str(n-i)+'_'+file['href'].split('/')[-1]
          
          # export data to local path
          with open(REPORTS_PATH/file_name, 'wb') as f: 
              f.write(result.content)
          
          # Wait five second before next iteration 
          time.sleep(1)           

else: 
  print(f'[Fail] Status error:{response.status_code}')

In [41]:
# # Set the locale to Spanish for month names
# locale.setlocale(locale.LC_TIME, 'es_ES.utf8')

# Getting link for each year
link_years = soup.find_all(lambda tag: tag.name == 'a' and any(substring in tag.get('title', '').lower() for substring in ['boletín mayorista semanal','boletin mayorista semanal', 'mayoristas boletín semanal']))

# within each year
for link in link_years:
    
    print(f' Working on {link.text.strip()} data')

    # Request for reports available in such year
    r = requests.get(url_base + link['href'], headers = headers)
    soup_year = BeautifulSoup(r.content, "html.parser")
    
    # Create links for each report within the given year
    target_links = [item for item in soup_year.find_all(target = '_blank') if 'Anexo' in item.text]
    date_elements = soup_year.find_all(lambda tag: tag.name == 'td' and tag.find('img', src="/files/images/boton.png")) #
    

    n = len(target_links)

    # Create folders per year
    REPORTS_PATH = Path.cwd()/'reports'/link.text.strip()
    if REPORTS_PATH.exists():
        print(f'{REPORTS_PATH} already exists')
    else: 
        REPORTS_PATH.mkdir(parents= True, exist_ok=True)

    
    # For each link, create file in local and fill it with data 
    for i, file in tqdm(enumerate(target_links)):
        # print(file)
        # request information
        try: 
            result = requests.get(url_base+file['href'], headers = headers)
        except:
            result = requests.get(file['href'], headers = headers)
        
        # name output file
        file_name = 'week_'+str(n-i)+'_'+file['href'].split('/')[-1]
        
        # export data to local path
        with open(REPORTS_PATH/file_name, 'wb') as f: 
            f.write(result.content)
        
        # Wait five second before next iteration 
        time.sleep(1)


NameError: name 'locale' is not defined

In [78]:
def first_format_wrangling(path_list):
    # Creating target dataframe
    year_file = pd.DataFrame()

    # Creating food categories 
    categories_dict = {1: 'verduras_hortalizas',
        2: 'frutas_frescas',
        3: 'tuberculos_raices_platanos',
        4: 'granos_cereales',
        5: 'huevos_lacteos',
        6: 'carnes',
        7: 'pescados',
        8: 'productos_procesados'}

    # Iterating over list of paths 
    for file_path in tqdm(path_list):
        
        # Capturing off cases
        try: 
            # Import data
            with open(file_path, 'rb') as f: 
                dataframe = pd.read_excel(f, header = None)
            
            # Keeping only first five columns and renaming them
            dataframe = dataframe.iloc[:,0:5]
            dataframe.columns = ['ciudad','precio_minimo','precio_maximo','precio_medio', 'tendencia']
            dataframe['ciudad'] = dataframe['ciudad'].str.lower().str.replace('bogotá, d.c.', 'bogota')
            
            # rows where ciudad is non null
            dataframe = dataframe[~dataframe['ciudad'].isnull()]
            
            # This formatting would have eight food categories within the same spreadsheet divided only by a big title. 
            # Such title would include the word 'cuadro'. So, to separate categories, we look for blocks of data contained 
            # within two consecutive appareances of such words. 

            # Get row indexes where the word 'cuadro' is present
            index_cuadro = dataframe[dataframe['ciudad'].str.contains('cuadro')].index

            # Creating target dataframe for all data
            df_final= pd.DataFrame()

            # Itearting over food categories. 
            for i_categoria in range(len(index_cuadro)):
                
                # Capturing first seven categories
                if i_categoria < 7:
                    dataframe_categoria = dataframe[index_cuadro[i_categoria]+2:index_cuadro[i_categoria+1]]
                # Capturing last category
                else: 
                    dataframe_categoria = dataframe[index_cuadro[i_categoria]+2:]

                # Within each category block, add category name
                dataframe_categoria['categoria'] = categories_dict[i_categoria+1]

                # within each category block, there are several products. In the whole reporting, products are very likely to contain 
                # several rows (same food item in different locations). What identifies such product blocks is the fact that the precio_minimo
                # column will be blank. So the product data would be contain within two consecutive occurrencies of blank prices. 
                index_producto = dataframe_categoria[dataframe_categoria['precio_minimo'].isnull()].index

                # creating target data frame for product category
                df_categoria_final = pd.DataFrame()

                # Iterating over products within food category
                for i_producto in range(len(index_producto)): 
                    
                    # Capturing the first product in the category 
                    if i_producto == 0:
                        dataframe_producto = dataframe_categoria.loc[index_producto[i_producto]-1:index_producto[i_producto+1]-1].reset_index(drop = True)
                    
                    # Capturing all intermediate products
                    elif i_producto < len(index_producto)-1: 
                        dataframe_producto = dataframe_categoria.loc[index_producto[i_producto]:index_producto[i_producto+1]-1].reset_index(drop = True)
                    
                    # Capturing last product within category 
                    else: 
                        dataframe_producto = dataframe_categoria.loc[index_producto[i_producto]:].reset_index(drop = True)
                    
                    # Adding product name column to each block of products
                    dataframe_producto['producto'] = dataframe_producto['ciudad'][0]

                    # Keeping only city name under the ciudad column
                    dataframe_producto['ciudad'] = dataframe_producto['ciudad'].str.replace(r'\s*\([^)]*\)', '', regex=True)
                    
                    # The name of the marketplaces is included on some of the city names. So we try to retrieve it
                    try: 
                        dataframe_producto['mercado'] = dataframe_producto['ciudad'].str.split(',').str[1].str.strip()
                    except:
                        dataframe_producto['mercado'] = np.nan
                    
                    # Getting a clean version of city name
                    try: 
                        dataframe_producto['ciudad'] = dataframe_producto['ciudad'].str.split(',').str[0].str.strip()
                    except: 
                        None
                    # Dropping first row
                    dataframe_producto = dataframe_producto.drop(0)

                    # Putting together all data for products within food category 
                    df_categoria_final = pd.concat([df_categoria_final, dataframe_producto], ignore_index = True)

                # Putting together all data 
                df_final = pd.concat([df_final,df_categoria_final], ignore_index = True)

            # Once data per file is complete, time stamps are added: year and week number
            df_final['semana_no'] = int(file_path.name.split('_')[1])#file_path.stem[5:7]
            df_final['anho'] = file_path.stem[-4:]
            
            # Adding data file to main target dataframe
            year_file = pd.concat([year_file,df_final], ignore_index = True)
            year_file = year_file[['anho','semana_no','categoria','producto','ciudad','precio_minimo','precio_maximo','precio_medio','tendencia']]
        
        # Printing what went wrong
        except Exception as e: 
            print(f'[Info] There is a problem in {file_path}: \n {e}')
    
    return year_file;

In [79]:
def second_format_paths():
    # Overall files are consistent with formating, but there are two major types of formats. First of them would apply up to week 19 of 2018. 
    first_format = ['2018', '2019', '2020', '2021', '2022', '2023']
    final_files_paths = []

    # Putting together overall list of files
    for year in first_format: 
        
        # Collecting paths from each year
        root_path = Path.cwd()/'reports'/year
        
        # collecting both xls and xlsx files
        xls_files = list(root_path.glob('*.xls'))
        xlsx_files = list(root_path.glob('*.xlsx'))
        all_files = xls_files + xlsx_files
        
        # This formatting would only apply after the 20th week of 2018
        if year == '2018':
            all_files = [path for path in all_files if int(path.stem.split('_')[1]) > 19]
        
        # Putting together the list of paths
        final_files_paths.extend(all_files)
    
    return final_files_paths;

In [91]:
def second_format_wrangling(path_list):
    
    # Creating target dataframe
    year_file2 = pd.DataFrame()

    # Creating food categories 
    categories_dict = {1: 'verduras_hortalizas',
        2: 'frutas_frescas',
        3: 'tuberculos_raices_platanos',
        4: 'granos_cereales',
        5: 'huevos_lacteos',
        6: 'carnes',
        7: 'pescados',
        8: 'productos_procesados'}

    # Iteration over list of paths
    for file_path in tqdm(path_list): 

        # Capturing off cases
        try: 
            # Importing file and extracting book names
            xl = pd.ExcelFile(file_path)
            ref_dict = {i: xl.sheet_names[i] for i in range(len(xl.sheet_names))}
            
            # Creating target dataframe for file data
            dataframe_file = pd.DataFrame()

            # Iterating over tabs within file
            for index in range(1,9):
                
                # Importing file
                with open(file_path, 'rb') as f: 
                    dataframe = pd.read_excel(f, sheet_name = ref_dict[index])

                # within this second type of formatting, there is two groups based on a subtle
                # detail: In one, data would start at row 10, in the other, data would start at row 11
                if pd.isnull(dataframe.iloc[9,0]):
                    dataframe = dataframe.iloc[10:,:6]
                else: 
                    dataframe = dataframe.iloc[9:,:6]

                # Setting column names
                dataframe.columns = ['producto', 'ciudad','precio_minimo','precio_maximo','precio_medio', 'tendencia']
                # keeping rows non null for ciudad column only
                dataframe = dataframe[~dataframe['ciudad'].isnull()]

                # Adding categoria and ciudad info 
                dataframe['categoria'] = categories_dict[index]
                dataframe['ciudad'] = dataframe['ciudad'].str.lower().str.replace('bogotá, d.c.', 'bogota')
                dataframe['ciudad'] = dataframe['ciudad'].str.replace(r'\s*\([^)]*\)', '', regex=True)
                                
                # The name of the marketplaces is included on some of the city names. So we try to retrieve it
                try: 
                    dataframe['mercado'] = dataframe['ciudad'].str.split(',').str[1].str.strip()
                except:
                    dataframe['mercado'] = np.nan
                                
                # Getting a clean version of city name
                try: 
                    dataframe['ciudad'] = dataframe['ciudad'].str.split(',').str[0].str.strip()
                except: 
                    pass
                
                # Once data per file is complete, time stamps are added: year and week number
                dataframe['semana_no'] = int(file_path.name.split('_')[1])#file_path.stem[5:7]
                dataframe['anho'] = file_path.stem[-4:]

                 # Adding data file to main target dataframe
                dataframe = dataframe[['anho','semana_no','categoria','producto','ciudad','precio_minimo','precio_maximo','precio_medio','tendencia']]
                dataframe_file = pd.concat([dataframe_file, dataframe], ignore_index = True)

            # Adding file data to target dataframe 
            year_file2 = pd.concat([year_file2, dataframe_file], ignore_index = True)
        
        # Printing what went wrong
        except Exception as e: 
            print(f'[Info] There is a problem in {file_path}: \n {e}')

    return year_file2;

In [101]:
def data_transformation():

    # Starting time counter
    start_time = time.perf_counter()
    
    # First format data (All data within same tab)
    print('Working on first format batch')
    first_batch = first_format_wrangling(first_format_paths())

    # Second format data (with tabs within spreadsheets)
    print('Working on second format batch')
    second_batch = second_format_wrangling(second_format_paths())

    # Merging resulting data
    final_dataframe = pd.concat([first_batch, second_batch], ignore_index = True)
    
    # Ending time counter
    end_time = time.perf_counter()
    
    # Computing total processing time 
    total_time = end_time - start_time
    print(f'[Info] Total execution time {total_time:.2f}')

    return final_dataframe;

In [102]:
full_sipsa = data_transformation()

Working on first format batch


  0%|          | 0/285 [00:00<?, ?it/s]

100%|██████████| 285/285 [19:56<00:00,  4.20s/it]


Working on second format batch


100%|██████████| 276/276 [14:39<00:00,  3.19s/it]


[Info] Total execution time 2077.61


In [13]:
full_sipsa = pd.read_csv('reports/full_report.csv')

In [14]:
full_sipsa

,anho,semana_no,categoria,producto,ciudad,precio_minimo,precio_maximo,precio_medio,tendencia,region
0,2012,1,verduras_hortalizas,acelga,barranquilla,2867,3000.0,2922,+,caribe
1,2012,1,verduras_hortalizas,acelga,bogota,300,333.0,303,--,NaN
2,2012,1,verduras_hortalizas,acelga,bucaramanga,1000,1867.0,1296,--,andina
3,2012,1,verduras_hortalizas,acelga,cali,1050,1500.0,1281,+++,pacífico
4,2012,1,verduras_hortalizas,acelga,cali,1050,1433.0,1242,+++,pacífico
...,...,...,...,...,...,...,...,...,...,...
2537469,2023,9,productos_procesados,Vinagre,pamplona,1750,1833.0,1778,=,NaN
2537470,2023,9,productos_procesados,Vinagre,pereira,6216,6840.0,6588,-,andina
2537471,2023,9,productos_procesados,Vinagre,santa marta,1525,1583.0,1553,+,caribe
2537472,2023,9,productos_procesados,Vinagre,sincelejo,1333,1875.0,1542,-,caribe


In [18]:
full_sipsa.to_csv('reports/full_report.csv', index = False)

city_to_region = {
    'Barranquilla': 'Caribe',
    'Cartagena': 'Caribe',
    'Santa Marta': 'Caribe',
    'Valledupar': 'Caribe',
    'Montería': 'Caribe',
    'Sincelejo': 'Caribe',
    'Riohacha': 'Caribe',
    'Ciénaga': 'Caribe',
    'Magangué': 'Caribe',
    'Maicao': 'Caribe',
    'Turbo': 'Caribe',
    'Lorica': 'Caribe',
    'Sahagún': 'Caribe',
    'Aracataca': 'Caribe',
    'El Banco': 'Caribe',
    'Girardot': 'Andina',
    'Ibagué': 'Andina',
    'Neiva': 'Andina',
    'Pereira': 'Andina',
    'Manizales': 'Andina',
    'Armenia': 'Andina',
    'Cali': 'Pacífico',
    'Buenaventura': 'Pacífico',
    'Tuluá': 'Pacífico',
    'Palmira': 'Pacífico',
    'Pasto': 'Pacífico',
    'Popayán': 'Pacífico',
    'Tumaco': 'Pacífico',
    'Yumbo': 'Pacífico',
    'Quibdó': 'Pacífico',
    'Bogotá': 'Andina',
    'Medellín': 'Andina',
    'Bucaramanga': 'Andina',
    'Cúcuta': 'Andina',
    'Villavicencio': 'Orinoquía',
    'Yopal': 'Orinoquía',
    'Arauca': 'Orinoquía',
    'Florencia': 'Amazonía',
    'Mocoa': 'Amazonía',
    'Leticia': 'Amazonía',
    'Puerto Carreño': 'Orinoquía',
    'Mitú': 'Amazonía',
    'Inírida': 'Amazonía',
}


In [24]:

full_sipsa.loc[full_sipsa['region'].isnull(), 'ciudad'].unique()


array(['bogota', 'duitama', 'ipiales', 'pamplona', 'sogamoso', 'tunja',
       'cartago', 'rionegro', 'san gil', 'socorro', 'chiquinquirá',
       'el santuario', 'marinilla', 'cajamarca', 'carmen de viboral',
       'la ceja', 'san vicente', 'sonsón', 'peñol', 'santa bárbara',
       'yarumal', 'la virginia', 'la unión', 'la parada', 'la dorada',
       'charalá', 'güepsa', 'moniquirá', 'puente nacional', 'santana',
       'vélez', 'caparrapí', 'nocaima', 'villeta', 'honda', 'ubaté',
       'cartagena frigorífico candelaria', 'san marcos', 'alvarado',
       'espinal', 'lérida', 'purificación', 'venadillo', 'cereté',
       'san gilpanela', 'yolombó', 'malambo', 'el carmen de viboral',
       'túquerres', 'san andrés de tumaco', 'ancuyá', 'consacá',
       'sandoná', 'ancuya', 'tibasosa', 'san sebastián de mariquita'],
      dtype=object)

In [12]:
df['region'] = df['ciudad'].map(city_to_region)

0            caribe
1               NaN
2            andina
3          pacífico
4          pacífico
             ...   
2523963         NaN
2523964      andina
2523965      caribe
2523966      caribe
2523967      caribe
Name: ciudad, Length: 2523968, dtype: object

In [11]:
city_to_region = {
    'barranquilla': 'caribe',
    'cartagena': 'caribe',
    'santa marta': 'caribe',
    'valledupar': 'caribe',
    'montería': 'caribe',
    'sincelejo': 'caribe',
    'riohacha': 'caribe',
    'ciénaga': 'caribe',
    'magangué': 'caribe',
    'maicao': 'caribe',
    'turbo': 'caribe',
    'lorica': 'caribe',
    'sahagún': 'caribe',
    'aracataca': 'caribe',
    'el banco': 'caribe',
    'girardot': 'andina',
    'ibagué': 'andina',
    'neiva': 'andina',
    'pereira': 'andina',
    'manizales': 'andina',
    'armenia': 'andina',
    'cali': 'pacífico',
    'buenaventura': 'pacífico',
    'tuluá': 'pacífico',
    'palmira': 'pacífico',
    'pasto': 'pacífico',
    'popayán': 'pacífico',
    'tumaco': 'pacífico',
    'yumbo': 'pacífico',
    'quibdó': 'pacífico',
    'bogotá': 'andina',
    'medellín': 'andina',
    'bucaramanga': 'andina',
    'cúcuta': 'andina',
    'villavicencio': 'orinoquía',
    'yopal': 'orinoquía',
    'arauca': 'orinoquía',
    'florencia': 'amazonía',
    'mocoa': 'amazonía',
    'leticia': 'amazonía',
    'puerto carreño': 'orinoquía',
    'mitú': 'amazonía',
    'inírida': 'amazonía',
}
